In [13]:
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as pt
%matplotlib inline

In [17]:
class Discriminator(nn.Module):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_layers= nn.Sequential(nn.Conv2d(1, 32, (3,3), padding=1),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2)),
                                 nn.Conv2d(32, 64, (3,3), padding=1),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2,2)))
        self.linear_layers = nn.Sequential(
                                 nn.Linear(3136, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, 10),
                                 nn.Softmax()
                                 # nn.MaxPool2d((2,2))
                                 )
    
    def forward(self, inp):
        conv_output = self.conv_layers(inp)
        return self.linear_layers(conv_output.view(conv_output.size(0), -1))

In [40]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(1, 512, (5, 5)), # 14
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, (5, 5)), # 18
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, (3, 3)), # 20
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, (3, 3)), # 22
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, (3, 3)), # 24
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, (3, 3)), # 26
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, (3, 3)), # 28
            nn.ReLU(),            
        )
    
    def forward(self, z):
        return self.net(z)

In [39]:
# inp = Variable(torch.rand( 1, 1, 28, 28))
d = Discriminator()
num_epochs = 100
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor()])))
count =0
for batch, (train_data, target) in enumerate(train_loader):
    if count == 100:
        break
    count +=1
    # print("train : ", train_data)
    # print("target : ", target)
    for epoch in range(num_epochs):
        output = d.forward(Variable(train_data))
        optimizer = optim.Adam(d.parameters())
        # target = Variable(target)
        target_t = torch.zeros(1, 10)
        # print(target_t[0][target-1])
        target_t[0][target] = 1
        # print("size ", target.size())
        loss_obj = nn.MSELoss()
        loss = loss_obj(output, Variable(target_t))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(" epoch : ", epoch, "loss : ", loss.data[0])

In [41]:
g = Generator()
z = torch.rand(1,1,10,10)
print(g.forward(Variable(z)))

TypeError: argument 0 is not a Variable